IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

# remove warnings
import warnings
warnings.filterwarnings('ignore')

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\FAULT_M1_10


In [3]:
guasto = 2

VOLO M1 FAULT 10%

In [4]:
rcou_m1_fault10 = pd.read_csv("RCOU.csv")
rcou_m1_fault10 = rcou_m1_fault10.astype("float64")
rcou_m1_fault10 = rcou_m1_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1_fault10 = rcou_m1_fault10[((rcou_m1_fault10['C9'] >= potenza) & (rcou_m1_fault10['C10'] >= potenza) & (rcou_m1_fault10['C11'] >= potenza) & (rcou_m1_fault10['C12'] >= potenza) & (rcou_m1_fault10['C13'] >= potenza) & (rcou_m1_fault10['C14']>= potenza))]

rcou_m1_fault10 = rcou_m1_fault10.reset_index(drop=True)


display(rcou_m1_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,1.648547e+09,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0
1,1.648559e+09,1351.0,1504.0,1451.0,1414.0,1366.0,1493.0
2,1.648580e+09,1467.0,1420.0,1429.0,1459.0,1448.0,1441.0
3,1.648601e+09,1525.0,1374.0,1393.0,1511.0,1466.0,1443.0
4,1.648612e+09,1436.0,1483.0,1442.0,1478.0,1413.0,1504.0
...,...,...,...,...,...,...,...
7959,1.749460e+09,1576.0,1423.0,1531.0,1477.0,1501.0,1508.0
7960,1.749470e+09,1555.0,1447.0,1515.0,1492.0,1459.0,1545.0
7961,1.749481e+09,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0
7962,1.749491e+09,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0


In [5]:
min=rcou_m1_fault10['TimeUS'][0]
max=rcou_m1_fault10['TimeUS'][len(rcou_m1_fault10)-1]
print(max)
print(min)

1749502709.0
1648547241.0


In [6]:
xkf1_m1_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault10 = xkf1_m1_fault10.astype("float64")
xkf1_m1_fault10 = xkf1_m1_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault10 = xkf1_m1_fault10[((xkf1_m1_fault10['TimeUS'] >= min) & (xkf1_m1_fault10['TimeUS'] <= max))]
xkf1_m1_fault10 = xkf1_m1_fault10.reset_index(drop=True)
print(xkf1_m1_fault10)

            TimeUS  Roll  Pitch    Yaw
0     1.648557e+09 -1.24   1.59  19.96
1     1.648568e+09 -1.34   1.57  19.96
2     1.648577e+09 -1.25   1.57  19.94
3     1.648588e+09 -1.13   1.58  19.94
4     1.648599e+09 -1.03   1.62  19.93
...            ...   ...    ...    ...
9157  1.749457e+09 -1.53   0.06   6.15
9158  1.749468e+09 -1.50   0.07   6.16
9159  1.749478e+09 -1.41   0.00   6.13
9160  1.749489e+09 -1.21  -0.05   6.14
9161  1.749500e+09 -0.92  -0.12   6.20

[9162 rows x 4 columns]


In [7]:
att_m1_fault10 = pd.read_csv("ATT.csv")
att_m1_fault10 = att_m1_fault10.astype("float64")
att_m1_fault10 = att_m1_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault10 = att_m1_fault10[((att_m1_fault10['TimeUS'] >= min) & (att_m1_fault10['TimeUS'] <= max))]
att_m1_fault10=att_m1_fault10.reset_index(drop=True)
print(att_m1_fault10)

            TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     1.648558e+09    -0.06 -1.24     -0.40   1.59   19.79  19.96
1     1.648569e+09    -0.05 -1.34     -0.41   1.57   19.79  19.96
2     1.648579e+09    -0.04 -1.25     -0.41   1.57   19.79  19.94
3     1.648590e+09    -0.04 -1.13     -0.42   1.58   19.79  19.94
4     1.648600e+09    -0.04 -1.03     -0.43   1.62   19.79  19.93
...            ...      ...   ...       ...    ...     ...    ...
9158  1.749459e+09     0.74 -1.53     -0.32   0.06    8.12   6.15
9159  1.749470e+09     0.75 -1.50     -0.34   0.07    8.12   6.16
9160  1.749480e+09     0.76 -1.41     -0.34   0.00    8.12   6.13
9161  1.749490e+09     0.75 -1.21     -0.33  -0.05    8.12   6.14
9162  1.749502e+09     0.74 -0.92     -0.32  -0.12    8.12   6.20

[9163 rows x 7 columns]


In [8]:
if len(att_m1_fault10)>len(xkf1_m1_fault10):
    to_add=att_m1_fault10[len(xkf1_m1_fault10):]
    att_m1_fault10=att_m1_fault10[:len(xkf1_m1_fault10)]

if len(xkf1_m1_fault10)>len(att_m1_fault10):
    to_add=xkf1_m1_fault10[len(att_m1_fault10):]

for idx,i in enumerate(att_m1_fault10['Roll']):
    if(xkf1_m1_fault10['Roll'][idx] != i):
        att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Pitch']):
    if(xkf1_m1_fault10['Pitch'][idx] != i):
        att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Yaw']):
    if(xkf1_m1_fault10['Yaw'][idx] != i):
        att_m1_fault10['Yaw'][idx] = ((att_m1_fault10['Yaw'][idx] + xkf1_m1_fault10['Yaw'][idx])/2)

att_m1_fault10=pd.concat([att_m1_fault10,to_add])

print(att_m1_fault10)

            TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0     1.648558e+09    -0.06 -1.240     -0.40  1.590   19.79  19.960
1     1.648569e+09    -0.05 -1.340     -0.41  1.570   19.79  19.960
2     1.648579e+09    -0.04 -1.250     -0.41  1.570   19.79  19.940
3     1.648590e+09    -0.04 -1.130     -0.42  1.580   19.79  19.940
4     1.648600e+09    -0.04 -1.030     -0.43  1.620   19.79  19.930
...            ...      ...    ...       ...    ...     ...     ...
9158  1.749459e+09     0.74 -1.515     -0.32  0.065    8.12   6.155
9159  1.749470e+09     0.75 -1.455     -0.34  0.035    8.12   6.145
9160  1.749480e+09     0.76 -1.310     -0.34 -0.025    8.12   6.135
9161  1.749490e+09     0.75 -1.065     -0.33 -0.085    8.12   6.170
9162  1.749502e+09     0.74 -0.920     -0.32 -0.120    8.12   6.200

[9163 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault10 = esc_0_m1_fault10.astype("float64")
esc_0_m1_fault10 = esc_0_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault10 = esc_0_m1_fault10[((esc_0_m1_fault10['TimeUS'] >= min) & (esc_0_m1_fault10['TimeUS'] <= max))]
esc_0_m1_fault10=esc_0_m1_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault10 = esc_1_m1_fault10.astype("float64")
esc_1_m1_fault10 = esc_1_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault10 = esc_1_m1_fault10[((esc_1_m1_fault10['TimeUS'] >= min) & (esc_1_m1_fault10['TimeUS'] <= max))]
esc_1_m1_fault10=esc_1_m1_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault10 = esc_2_m1_fault10.astype("float64")
esc_2_m1_fault10 = esc_2_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault10 = esc_2_m1_fault10[((esc_2_m1_fault10['TimeUS'] >= min) & (esc_2_m1_fault10['TimeUS'] <= max))]
esc_2_m1_fault10=esc_2_m1_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault10 = esc_3_m1_fault10.astype("float64")
esc_3_m1_fault10 = esc_3_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault10 = esc_3_m1_fault10[((esc_3_m1_fault10['TimeUS'] >= min) & (esc_3_m1_fault10['TimeUS'] <= max))]
esc_3_m1_fault10=esc_3_m1_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault10 = esc_4_m1_fault10.astype("float64")
esc_4_m1_fault10 = esc_4_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault10 = esc_4_m1_fault10[((esc_4_m1_fault10['TimeUS'] >= min) & (esc_4_m1_fault10['TimeUS'] <= max))]
esc_4_m1_fault10=esc_4_m1_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault10 = esc_5_m1_fault10.astype("float64")
esc_5_m1_fault10 = esc_5_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault10 = esc_5_m1_fault10[((esc_5_m1_fault10['TimeUS'] >= min) & (esc_5_m1_fault10['TimeUS'] <= max))]
esc_5_m1_fault10=esc_5_m1_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m1_fault10 = pd.read_csv("ESC_0.csv")\nesc_0_m1_fault10 = esc_0_m1_fault10.astype("float64")\nesc_0_m1_fault10 = esc_0_m1_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m1_fault10 = esc_0_m1_fault10[((esc_0_m1_fault10[\'TimeUS\'] >= min) & (esc_0_m1_fault10[\'TimeUS\'] <= max))]\nesc_0_m1_fault10=esc_0_m1_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m1_fault10 = pd.read_csv("ESC_1.csv")\nesc_1_m1_fault10 = esc_1_m1_fault10.astype("float64")\nesc_1_m1_fault10 = esc_1_m1_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m1_fault10 = esc_1_m1_fault10[((esc_1_m1_fault10[\'TimeUS\'] >= min) & (esc_1_m1_fault10[\'TimeUS\'] <= max))]\nesc_1_m1_fault10=esc_1_m1_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault10 = imu_0_m1_fault10.astype("float64")
imu_0_m1_fault10 = imu_0_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault10 = imu_0_m1_fault10[((imu_0_m1_fault10['TimeUS'] >= min) & (imu_0_m1_fault10['TimeUS'] <= max))]

imu_1_m1_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault10 = imu_1_m1_fault10.astype("float64")
imu_1_m1_fault10 = imu_1_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault10 = imu_1_m1_fault10[((imu_1_m1_fault10['TimeUS'] >= min) & (imu_1_m1_fault10['TimeUS'] <= max))]

imu_2_m1_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault10 = imu_2_m1_fault10.astype("float64")
imu_2_m1_fault10 = imu_2_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault10 = imu_2_m1_fault10[((imu_2_m1_fault10['TimeUS'] >= min) & (imu_2_m1_fault10['TimeUS'] <= max))]

imu_m1_fault10 = pd.concat((imu_0_m1_fault10, imu_1_m1_fault10, imu_2_m1_fault10))
imu_m1_fault10=imu_m1_fault10.groupby(imu_m1_fault10.TimeUS, as_index=False).mean()

display(imu_m1_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,1.648552e+09,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406
1,1.648558e+09,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221
2,1.648562e+09,0.041824,0.100781,0.001137,0.231455,0.761186,-9.726832
3,1.648564e+09,0.220820,0.071107,0.001237,0.379594,0.636334,-9.590402
4,1.648567e+09,0.112270,-0.002903,0.002685,0.414795,0.359073,-9.528167
...,...,...,...,...,...,...,...
34809,1.749488e+09,0.280210,-0.035147,0.002372,0.501059,0.538888,-10.258573
34810,1.749490e+09,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337
34811,1.749493e+09,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707
34812,1.749499e+09,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault10 = pd.merge_ordered(imu_m1_fault10,att_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m1_fault10 = pd.merge_ordered(m1_fault10, esc_0_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_1_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_2_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_3_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_4_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_5_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")'''
m1_fault10 = pd.merge_ordered(m1_fault10, rcou_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m1_fault10["TimeUS"] = m1_fault10["TimeUS"] - m1_fault10.iloc[0]["TimeUS"]
m1_fault10["TimeUS"] = m1_fault10["TimeUS"].astype(int)
m1_fault10["Guasto"] = guasto

In [13]:
last_cell = m1_fault10.iloc[-1, m1_fault10.columns.get_loc('TimeUS')]
print(last_cell)

100955468


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

          TimeUS
0              0
1           2857
2           5714
3           8571
4          11428
...          ...
35332  100943524
35333  100946381
35334  100949238
35335  100952095
35336  100954952

[35337 rows x 1 columns]


In [16]:
m1_fault10_final = pd.merge_ordered(m1_fault10,df)
m1_fault10_final=m1_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m1_fault10_final)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0              0  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
1           2857  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
2           5164  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
3           5714  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
4           8571  0.038953  0.004357  0.030866  0.365539  0.468139  -9.389406   
...          ...       ...       ...       ...       ...       ...        ...   
87253  100952095  0.296264 -0.050650  0.081374 -0.083817  0.647380 -10.938693   
87254  100954357  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
87255  100954568  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
87256  100954952  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   
87257  100955468  0.355682 -0.062922  0.094976 -0.154238  0.797622 -11.020897   

       DesRoll   Roll  DesP

In [17]:
m1_fault10_final = m1_fault10_final[m1_fault10_final.TimeUS.isin(to_be)]

In [18]:
m1_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,2857,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,5714,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,8571,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
7,11428,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87247,100943524,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337,0.75,-1.065,-0.33,-0.085,8.12,6.17,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
87250,100946381,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
87251,100949238,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
87253,100952095,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


In [19]:
from datetime import timedelta
m1_fault10_final=m1_fault10_final.reset_index(drop=True)
m1_fault10_final['TimeUS'] = pd.to_datetime(m1_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
1,00:00:00.002857,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
2,00:00:00.005714,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
3,00:00:00.008571,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
4,00:00:00.011428,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221,-0.06,-1.240,-0.40,1.590,19.79,19.96,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35332,00:01:40.943524,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337,0.75,-1.065,-0.33,-0.085,8.12,6.17,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2.0
35333,00:01:40.946381,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35334,00:01:40.949238,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0
35335,00:01:40.952095,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693,0.75,-1.065,-0.33,-0.085,8.12,6.17,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault10_final)/350)):
        V1=m1_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':round(mean,20),f'var({V})':round(var,20),f'kurt({V})':round(kurt,20),f'RMS({V})':round(rms,20),f'freq1({V})':round(xf[index1],20),f'freq2({V})':round(xf[index2],20),f'amp1({V})':round(h_max_peak_1,20),f'amp2({V})':round(h_max_peak_2,20)}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.032100,0.017081,-0.399284,0.134398,78.0,96.0,3.596543,3.414090,-0.014483,0.028038,-0.930231,0.167832,0.0,77.0,30.934448,1.652123,0.158300,0.014169,-1.421637,0.197959,79.0,81.0,1.463315,1.322576,-0.016790,0.096882,0.043306,0.311268,0.0,106.0,32.245803,15.724635,0.663812,0.050827,-0.162425,0.700949,106.0,105.0,16.123267,6.087865,-10.545771,0.555601,-0.641326,10.572005,0.0,95.0,125.657078,9.558735,-0.690684,0.822899,0.259198,1.139122,88.0,84.0,2.421199,2.195279,-1.038632,3.999740,-1.548391,2.251023,0.0,126.0,219.164774,12.400463,22.969373,9.731288,-1.294680,23.179639,117.0,85.0,-0.421434,-2.069081,1564.310541,3738.854717,2.017608,1565.501732,94.0,93.0,1433.226115,751.605212,1454.849003,3139.065706,-1.180470,1455.924361,103.0,111.0,986.802386,944.782868,1538.774929,3181.014912,0.711130,1539.805258,104.0,97.0,786.761672,425.725311,1484.746439,2804.138380,-0.808859,1485.687766,110.0,111.0,1472.996730,1260.633758,1495.495726,2808.873553,-0.053468,1496.431869,106.0,102.0,1357.778490,1316.635778,1530.205128,1397.957802,-0.375963,1530.660547,116.0,115.0,1356.236798,982.813328,2
1,1.0,-0.032346,0.017054,-0.400010,0.134356,102.0,80.0,3.601798,3.359201,-0.014131,0.028001,-0.928239,0.167695,0.0,67.0,30.796057,2.112457,0.158055,0.014150,-1.421505,0.197715,83.0,85.0,1.347358,1.222977,-0.016522,0.096631,0.050202,0.310853,0.0,99.0,32.505395,15.842663,0.663532,0.050710,-0.157313,0.700600,114.0,118.0,16.196587,5.686090,-10.543524,0.555795,-0.637170,10.569774,0.0,95.0,125.382041,9.182466,-0.697784,0.838300,0.263650,1.150140,73.0,97.0,2.411245,2.328822,-1.038949,3.988380,-1.544033,2.248658,0.0,119.0,222.587826,12.004870,22.985256,9.792356,-1.302722,23.196692,114.0,84.0,-0.650352,-1.751181,1564.525568,3744.477977,2.015006,1565.718395,98.0,97.0,1417.303430,959.364312,1454.616477,3149.154486,-1.183728,1455.695471,110.0,108.0,920.052547,773.041068,1538.988636,3188.028361,0.706907,1540.021102,102.0,94.0,764.039235,335.971852,1484.539773,2811.183599,-0.815464,1485.483603,101.0,102.0,1483.850630,1325.524715,1495.340909,2809.307951,-0.069023,1496.277301,107.0,111.0,1520.868027,1347.166450,1530.380682,1404.823329,-0.386721,1530.838288,113.0,112.0,1351.732999,889.971776,2
2,2.0,-0.032590,0.017026,-0.400594,0.134314,98.0,79.0,3.785400,3.155365,-0.013642,0.028006,-0.929273,0.167669,0.0,67.0,30.704383,2.061509,0.157845,0.014126,-1.420411,0.197485,79.0,81.0,1.174580,1.120307,-0.016336,0.096369,0.057969,0.310424,0.0,102.0,32.733791,15.841505,0.663447,0.050568,-0.149492,0.700419,103.0,101.0,16.292173,6.905085,-10.540989,0.556485,-0.633771,10.567277,0.0,94.0,125.207175,8.007901,-0.705099,0.854808,0.275912,1.161702,87.0,86.0,2.664270,2.156209,-1.039065,3.977055,-1.539754,2.246207,0.0,128.0,226.056739,11.651650,23.001190,9.854162,-1.3

In [24]:
m1_fault10 = df_merged

In [25]:
path_file = path_file.replace(r"M1\FAULT_M1_10", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
